# Project: Conway's Game Of Life 


## Overview

Conway’s Game of Life is a classic example of a **cellular automaton**. It consists of a grid of cells, each of which can be **alive** or **dead**. The grid evolves in discrete steps according to these simple rules, applied to each cell based on its eight neighbors:

1. Any live cell with fewer than two live neighbors dies (underpopulation).  
2. Any live cell with two or three live neighbors remains alive (survival).  
3. Any live cell with more than three live neighbors dies (overpopulation).  
4. Any dead cell with exactly three live neighbors becomes a live cell (reproduction).  

A number of animated gifs and visuals can be seen on the [Wiki page](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

## Learning Objectives

- Practice building up a project incrementally, starting with a straightforward solution and progressively introducing optimisations.  
- Practice understanding the impact of code changes on performance in regards to execution time and memory footprint. 
- Practice designing and running targeted tests to verify correctness at each development stage.  
- Practice measuring and analysing performance using benchmarking and profiling tools.  

## Your Path Forward

Choose the path (or combination of paths) that best matches your interests, and document your experiments as you go.  

1. **Build the Naïve Solution**  
   Implement the simple Game of Life from from scratch by following the four substeps (initialisation, neighbor counting, state update, and visualisation). If you get stuck or want a reference, check out the “Naïve Implementation: Four Substeps” code below.

2. **Profile the Baseline**  
   Take your solution to the naïve implementation solution below and use tools like `@time`, `@btime` (BenchmarkTools), and `Profile.@profile` to identify time consuming computation. You can also profile a ready made naive solution avaiable within the [course notes](implemented_game_of_life.ipynb).
  
3. **Understanding Code Changes on Performance**  
   Starting from the baseline naive solution, experiment with different code changes, measure the impact of each change, and compare results.

4. **Follow Your Own Curiosity**  
   If neither profiling nor low-level tweaking appeals, explore something else that interests you:  
   - Build richer or interactive visualisations (e.g., with `Makie` or `Plotly`)  
   - Extend to custom rule sets or sparse/infinite grids  
   - Integrate a GUI
   - Anything else that fits your learning goals!

## Naïve Implementation: Four Substeps

Below is a simple version of Game of Life. 

### Substep 1: Initialise the Grid
**Goal:** Initialise a 100×100 boolean grid with random `true`/`false` values to represent the starting state of the Game of Life.  
```{admonition} Substep 1 Solution
:class: dropdown
```Julia
const N = 100  # grid size
# Create a random N×N Matrix of Bool
grid = rand(Bool, N, N)
```
```

### Substep 2: Neighbor Counting

**Goal:** Count the number of live neighbors surrounding cell `(i, j)` in a finite `Matrix{Bool}` grid.  
```{admonition} Substep 2 Solution
:class: dropdown
```Julia
function count_alive_neighbours(grid::Matrix{Bool}, i::Int, j::Int)::Int
    subset_i = max(1, i - 1):min(size(grid, 1), i + 1)
    subset_j = max(1, j - 1):min(size(grid, 2), j + 1)
    return sum(grid[subset_i, subset_j]) - grid[i, j]  # don't count subject cell itself
end
```
How do you think this could be made more efficient?
```

### Substep 3: Compute Next State

**Goal:** Compute and return the next generation of the Game of Life grid by applying Conway’s rules, using a fresh `Matrix{Bool}` for the updated state.  
```{admonition} Substep 3 Solution
:class: dropdown
```Julia
function next_state(grid::Matrix{Bool})
    # Get grid dimensions: N1 rows, N2 columns
    N1, N2 = size(grid)

    # Pre-allocate a new grid of the same size, initially all false (dead)
    newgrid = falses(N1, N2)

    # Loop over every cell position (i, j)
    for i in 1:N1, j in 1:N2
        # Count the live neighbors around (i, j)
        cnt = count_alive_neighbours(grid, i, j)


        # Apply the Game of Life rules:
        # - A live cell stays alive if it has 2 or 3 neighbors.
        # - A dead cell becomes alive if it has exactly 3 neighbors.
        # Otherwise it remains or becomes dead.
        newgrid[i,j] = (grid[i,j] && (cnt == 2 || cnt == 3)) ||
                       (!grid[i,j] && cnt == 3)
    end

    # Return the updated grid for the next generation
    return newgrid
end
```
Are there any opportunities for a boost in performance?
```

### Substep 4: Run Simulation

**Goal:** Simulate the Game of Life for a given number of steps.  

```{admonition} Substep 4 Solution
:class: dropdown
```Julia
function simulate(initial_grid::Matrix{Bool}, steps::Int)
    # Get grid dimensions
    N1, N2 = size(initial_grid)

    # Make 3D array with time along the third axis.
    # Starts with only a single time index corresponding to initial
    # grid.
    simulations = reshape(initial_grid, N1, N2, 1)
    
    for step in 1:steps
        # Current latest generation
        current = simulations[:, :, end]
        
        # Update simulations with next generation
        next_gen = next_state(current)
        simulations = cat(simulations, next_gen; dims=3)
    end
    
    return simulations
end
```
How do you think this could be made more performant?
```

### Substep 5: Visualize

**Goal:** Animate the evolution of the Game of Life grid and save the result as a GIF using `Plots`. 

```{admonition} Substep 5 Solution
:class: dropdown
```Julia
function visualize_simulations(
        simulations::Array{Bool, 3};
        print_interval_percent::Int=10
)
    # Compute how many steps correspond to one print interval:
    # at least 1, scaled by the given percentage of total steps
    n_steps = size(simulations, 3)
    step_interval = max(1, floor(Int, n_steps * print_interval_percent / 100))

    t_start = time()
    anim = @animate for step in axes(simulations, 3)
        # If we've reached a print interval, output progress stats
        if step % step_interval == 0
            pct     = step / n_steps * 100
            elapsed = time() - t_start
            avg     = elapsed / step
            remain  = avg * (n_steps - step)
            @printf("Progress: %3.0f%% | Elapsed: %5.1fs | Remaining: %5.1fs\n",
                    pct, elapsed, remain)
        end

        # Generate a heatmap frame for the current grid state
        heatmap(
            view(simulations, :, :, step);
            color = :grays,
            axis = false,
            legend = false,
            framestyle = :none,
            aspect_ratio = 1
        )
    end
    
    # Ensure the “data” directory exists
    mkpath("data")

    # Write the animation to GIF at 10 frames per second
    gif(anim, "data/life_naive.gif"; fps = 10)
    println("Saved animation to data/life_naive.gif")
    
    # Detect if running inside a Jupyter notebook
    in_notebook = isdefined(Main, :IJulia) && IJulia.inited
 
    if in_notebook
        # Display the GIF inline in the notebook
        display("image/gif", read("data/life_naive.gif"))
    else
        # Otherwise, instruct the user to open the files manually
        println("Not running in a notebook.")
        println("Open `data/life_naive.gif` in your OS image viewer to see the results.")
    end
end
```
```

To view the live implementation along with an animated GIF, please proceed to the next page of this course!

## Hint Sheet

Keep in mind that while we haven’t discussed these topics in class, they’re offered here as guidance to help you with this project research. 
```{admonition} Hint 1 
:class: dropdown
### Formatted Output with `Printf`

**Module**: `using Printf`

**Macro**: `@printf` for C-style formatted printing to `STDOUT`.

````julia
@printf("Progress: %3.0f%% | Elapsed: %5.1fs | Remaining: %5.1fs\n",
        pct, elapsed, remain)
````
```

```{admonition} Hint 2 
:class: dropdown
### Animations with `Plots.jl`

**Module**: `using Plots`

**Macro**: `@animate` collects plot frames in a loop, which can then be exported with `gif(anim, filepath; fps=10)`.

````julia 
anim = @animate for step in 1:steps
    heatmap(grid; color=:grays, axis=false, legend=false, framestyle=:none)
end
````
```
